In [8]:
offlineprofit = list()
onlineprofit = list()
onlineprofit_abbas = list()
onlineprofit_abbas2 = list()

In [9]:
import os
from datetime import datetime
import pandas as pd
from read_file import read_jobs

# Test instance optimal profits for reference
optimal_profits = {
    "test1": 133,
    "test2": 44,
    "test3": 30,
    "test4": 10,
    "test5": 0,
    "test6": 130,
    "test7": 70
}

def get_time_horizon(jobs):
    return max(job["d"] for job in jobs)   # deadline inclusive

def dp_schedule(jobs, test_case_name):
    n = len(jobs)
    jobs = sorted(jobs, key=lambda x: x["d"])

    from functools import lru_cache
    @lru_cache(None)
    def dp(i, used_mask):
        if i == n:
            return 0
        job = jobs[i]
        best = -10**9
        skip_profit = -job["l"] + dp(i+1, used_mask)
        best = max(best, skip_profit)
        available_slots = [t for t in range(job["r"], job["d"]+1) if not (used_mask >> t) & 1]
        if len(available_slots) >= job["p"]:
            new_mask = used_mask
            for t in available_slots[:job["p"]]:
                new_mask |= (1 << t)
            take_profit = job["w"] + dp(i+1, new_mask)
            best = max(best, take_profit)
        return best

    total_profit = dp(0, 0)
    assigned = {job["id"]: [] for job in jobs}
    status = {}
    scheduled_jobs = []

    def reconstruct(i, used_mask):
        if i == n:
            return
        job = jobs[i]
        skip_profit = -job["l"] + dp(i+1, used_mask)
        best = dp(i, used_mask)
        if best == skip_profit:
            status[job["id"]] = f"NOT done → -{job['l']}"
            job["assigned_slots"] = None
            scheduled_jobs.append(job)
            reconstruct(i+1, used_mask)
            return
        available_slots = [t for t in range(job["r"], job["d"]+1) if not (used_mask >> t) & 1]
        new_mask = used_mask
        slots = []
        for t in available_slots[:job["p"]]:
            new_mask |= (1 << t)
            assigned[job["id"]].append(t)
            slots.append(t)
        status[job["id"]] = f"DONE → +{job['w']}"
        job["assigned_slots"] = slots
        scheduled_jobs.append(job)
        reconstruct(i+1, new_mask)

    reconstruct(0, 0)

    # Pretty print
    print("Schedule results:")
    for job in jobs:
        slots = assigned[job["id"]]
        print(f"Job {job['id']} {status[job['id']]}, slots = {slots if slots else 'null'}")
    
    # Extract base test name and show optimal comparison
    base_test_name = test_case_name.replace('_offline', '').replace('_online', '')
    optimal = optimal_profits.get(base_test_name, 'N/A')
    print(f"\nTotal profit: {total_profit} | Optimal: {optimal}")

    # Save results
    save_results_txt(test_case_name, scheduled_jobs, total_profit)
    log_results_csv(test_case_name, scheduled_jobs, total_profit)

    return assigned, total_profit


# ---------------------------
# Save results to CSV
# ---------------------------
def log_results_csv(test_case_name, scheduled_jobs, total_profit, csv_file="results_log.csv"):
    job_details = []
    for job in scheduled_jobs:
        slots = ",".join(map(str, job.get("assigned_slots", []))) if job.get("assigned_slots") else "null"
        job_details.append(f"id:{job['id']} r:{job['r']} d:{job['d']} p:{job['p']} w:{job['w']} l:{job['l']} slots:{slots}")
    log_data = {
        "date": datetime.now().date(),
        "time": datetime.now().time().strftime("%H:%M:%S"),
        "test_case": test_case_name,
        "total_profit": total_profit,
        "job_details": " | ".join(job_details)
    }
    df_log = pd.DataFrame([log_data])
    if os.path.exists(csv_file):
        df_log.to_csv(csv_file, mode="a", index=False, header=False)
    else:
        df_log.to_csv(csv_file, index=False, header=True)

# ---------------------------
# Save results to txt file
# ---------------------------
def save_results_txt(test_case_name, scheduled_jobs, total_profit, output_folder="results"):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    output_path = os.path.join(output_folder, f"{test_case_name}_offline.txt")
    with open(output_path, "w") as f:
        for job in sorted(scheduled_jobs, key=lambda x: x["id"]):
            if job.get("assigned_slots"):
                f.write(",".join(map(str, job["assigned_slots"])) + "\n")
            else:
                f.write("null\n")
        f.write(str(total_profit) + "\n")
        offlineprofit.append(total_profit)

    print(f"\nResults saved to {output_path}")


if __name__ == "__main__":
    #test_cases = ["test1", "test2", "test3", "test4", "test5", "test6", "test7"]
    
    test_cases = os.listdir("C:/Users/Collin/Documents/Universiteit Utrecht/Periode 1/Algorithms for Decision Support/Project/job_scheduling_instances")
    for test_case in test_cases:
        jobs = read_jobs(f"job_scheduling_instances/{test_case}")
        assigned, profit = dp_schedule(jobs, test_case)
        print("\n" + "-"*50 + "\n")


Schedule results:
Job 1 DONE → +64, slots = [12, 13, 14, 15, 16]
Job 7 DONE → +62, slots = [17, 18, 19, 20, 21]
Job 6 DONE → +97, slots = [4, 5, 6, 7, 8, 9, 10]
Job 3 DONE → +21, slots = [27, 28]
Job 2 DONE → +39, slots = [25, 26, 29, 30, 31, 32, 33, 34]
Job 4 DONE → +98, slots = [35, 36, 37, 38, 39]
Job 5 DONE → +32, slots = [62, 63, 64, 65]

Total profit: 413 | Optimal: N/A

Results saved to results\instance_0001.txt_offline.txt

--------------------------------------------------

Schedule results:
Job 8 DONE → +72, slots = [3, 4, 5]
Job 5 DONE → +44, slots = [6, 7, 8, 9, 10]
Job 1 DONE → +72, slots = [30, 31, 32, 33]
Job 10 DONE → +94, slots = [34, 35, 36, 37, 38, 39]
Job 15 DONE → +50, slots = [55, 56]
Job 17 DONE → +28, slots = [45]
Job 14 DONE → +39, slots = [40, 41, 42, 43, 44, 46, 47, 48, 49]
Job 9 DONE → +65, slots = [50, 51, 52, 53, 54, 57, 58, 59]
Job 6 NOT done → -29, slots = null
Job 2 DONE → +62, slots = [60, 61, 62, 63, 64, 65, 66, 67, 68, 69]
Job 11 DONE → +74, slots = 

In [10]:
import os
from datetime import datetime
import pandas as pd
from read_file import read_jobs

# Test instance optimal profits for reference
optimal_profits = {
    "test1": 133,
    "test2": 44,
    "test3": 30,
    "test4": 10,
    "test5": 0,
    "test6": 130,
    "test7": 70
}

# ---------------------------
# Online scheduler state
# ---------------------------
calendar = {}
scheduled_jobs = []
total_profit = 0

# ---------------------------
# Scheduler functions
# ---------------------------
def filter_infeasible(job):
    job["feasible"] = job["d"] - job["r"] + 1 >= job["p"]
    return job

def compute_score(job):
    job["score"] = (job["w"] + job["l"]) / job["p"] if job["feasible"] else -1
    return job

def schedule_job(job):
    global calendar, total_profit
    if not job["feasible"]:
        job["assigned_slots"] = None
        total_profit -= job["l"]
        print(f"Job {job['id']} NOT done → -{job['l']}, slots = null")
        scheduled_jobs.append(job)
        return job

    needed = job["p"]
    available_slots = [t for t in range(job["r"], job["d"] + 1) if calendar.get(t,0)==0]

    if len(available_slots) >= needed:
        job["assigned_slots"] = available_slots[:needed]
        for t in job["assigned_slots"]:
            calendar[t] = job["id"]
        total_profit += job["w"]
        print(f"Job {job['id']} DONE → +{job['w']}, slots = {job['assigned_slots']}")
    else:
        job["assigned_slots"] = None
        total_profit -= job["l"]
        print(f"Job {job['id']} NOT done → -{job['l']}, slots = null")

    scheduled_jobs.append(job)
    return job

# ---------------------------
# Save results to CSV
# ---------------------------
def log_results_csv(test_case_name, csv_file="results_log.csv"):
    global scheduled_jobs, total_profit
    job_details = []
    for job in scheduled_jobs:
        slots = ",".join(map(str, job["assigned_slots"])) if job["assigned_slots"] else "null"
        job_details.append(f"id:{job['id']} r:{job['r']} d:{job['d']} p:{job['p']} w:{job['w']} l:{job['l']} slots:{slots}")
    log_data = {
        "date": datetime.now().date(),
        "time": datetime.now().time().strftime("%H:%M:%S"),
        "test_case": test_case_name,
        "total_profit": total_profit,
        "job_details": " | ".join(job_details)
    }
    df_log = pd.DataFrame([log_data])
    if os.path.exists(csv_file):
        df_log.to_csv(csv_file, mode="a", index=False, header=False)
    else:
        df_log.to_csv(csv_file, index=False, header=True)

# ---------------------------
# Save results to txt file
# ---------------------------
def save_results_txt(test_case_name, output_folder="results"):
    global scheduled_jobs, total_profit
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    output_path = os.path.join(output_folder, f"{test_case_name}.txt")

    with open(output_path, "w") as f:
        for job in sorted(scheduled_jobs, key=lambda x: x["id"]):
            if job["assigned_slots"]:
                f.write(",".join(map(str, job["assigned_slots"])) + "\n")
            else:
                f.write("null\n")
        f.write(str(total_profit) + "\n")

    print(f"\nResults saved to {output_path}")



# ---------------------------
# Main online execution
# ---------------------------
def run_online_algorithm_from_file(input_file):
    global calendar, scheduled_jobs, total_profit
    # reset state
    calendar = {}
    scheduled_jobs = []
    total_profit = 0

    jobs = read_jobs(input_file)
    test_case_name = os.path.splitext(os.path.basename(input_file))[0] + "_online"
    print(f"Running online scheduling for {test_case_name}\n")

    for job in jobs:
        job = filter_infeasible(job)
        job = compute_score(job)
        schedule_job(job)
    
    # Extract base test name (without _online or .txt)
    base_test_name = os.path.splitext(os.path.basename(input_file))[0]
    optimal = optimal_profits.get(base_test_name.replace('_online','').replace('_offline',''), 'N/A')
    print(f"\nFinal total profit: {total_profit} | Optimal: {optimal}")
    onlineprofit.append(total_profit)

    log_results_csv(test_case_name)
    save_results_txt(test_case_name)

# ---------------------------
# Example usage
# ---------------------------
if __name__ == "__main__":
    #tests = ["test1.txt", "test2.txt", "test3.txt", "test4.txt", "test5.txt", "test6.txt", "test7.txt"]
    tests = os.listdir("C:/Users/Collin/Documents/Universiteit Utrecht/Periode 1/Algorithms for Decision Support/Project/job_scheduling_instances")

    for input_file in tests:
        run_online_algorithm_from_file(f'job_scheduling_instances/{input_file}')
        print("\n" + "-"*50 + "\n")


Running online scheduling for instance_0001_online

Job 1 DONE → +64, slots = [12, 13, 14, 15, 16]
Job 2 DONE → +39, slots = [25, 26, 27, 28, 29, 30, 31, 32]
Job 3 DONE → +21, slots = [33, 34]
Job 4 DONE → +98, slots = [35, 36, 37, 38, 39]
Job 5 DONE → +32, slots = [62, 63, 64, 65]
Job 6 DONE → +97, slots = [4, 5, 6, 7, 8, 9, 10]
Job 7 DONE → +62, slots = [17, 18, 19, 20, 21]

Final total profit: 413 | Optimal: N/A

Results saved to results\instance_0001_online.txt

--------------------------------------------------

Running online scheduling for instance_0002_online

Job 1 DONE → +72, slots = [30, 31, 32, 33]
Job 2 DONE → +62, slots = [46, 47, 48, 49, 50, 51, 52, 53, 54, 55]
Job 3 DONE → +32, slots = [88, 89, 90, 91]
Job 4 DONE → +78, slots = [77, 78, 79]
Job 5 DONE → +44, slots = [4, 5, 6, 7, 8]
Job 6 DONE → +39, slots = [56, 57, 58, 59, 60, 61, 62]
Job 7 DONE → +13, slots = [64, 65, 66, 67, 68]
Job 8 DONE → +72, slots = [3, 9, 10]
Job 9 NOT done → -14, slots = null
Job 10 DONE → +94

In [11]:
import os
from datetime import datetime
import pandas as pd
from read_file import read_jobs

# Optional: known optimal profits for printout
optimal_profits = {
    "test1": 133, "test2": 44, "test3": 30, "test4": 10, "test5": 0, "test6": 130, "test7": 70
}

# ---------------------------
# Config for the scoring exponents
# ---------------------------
A = 1.0  # exponent for (w + l)
B = 1.0  # exponent for p
C = 1.0  # exponent for frac_time_left
D = 1.0  # exponent for frac_work_left
EPS = 1e-12  # numerical safety

# ---------------------------
# Global state
# ---------------------------
calendar = {}           # t -> job_id (or 0 if idle)
scheduled_jobs = []     # list of annotated job dicts
total_profit = 0

# ---------------------------
# Helpers
# ---------------------------
def mark_infeasible(job):
    # Feasible if there are at least p usable slots in [r, d]
    job["feasible"] = (job["d"] - job["r"] + 1) >= job["p"]
    return job

def annotate_job(job):
    job["assigned_slots"] = []
    job["remaining"] = job["p"] if job["feasible"] else 0
    job["rejected"] = False
    return job

def frac_time_left(job, t):
    # proportion of window remaining at time t (inclusive)
    window = max(job["d"] - job["r"] + 1, 1)
    left = max(job["d"] - t + 1, 0)
    return max(left / window, EPS)

def frac_work_left(job):
    if job["p"] <= 0:
        return 1.0  # degenerate, but won't be scheduled
    return max(job["remaining"] / job["p"], EPS)

def dynamic_score(job, t):
    # (w+l)^A / (p^B * frac_time_left^C * frac_work_left^D)
    if job["remaining"] <= 0:
        return -float("inf")
    num = (job["w"] + job["l"]) ** A
    denom = (max(job["p"], EPS) ** B) * (frac_time_left(job, t) ** C) * (frac_work_left(job) ** D)
    return num / denom

def log_results_csv(test_case_name, csv_file="results_log.csv"):
    global scheduled_jobs, total_profit
    rows = []
    details = []
    for job in scheduled_jobs:
        slots = ",".join(map(str, job["assigned_slots"])) if job["assigned_slots"] else "null"
        details.append(
            f"id:{job['id']} r:{job['r']} d:{job['d']} p:{job['p']} w:{job['w']} l:{job['l']} slots:{slots}"
        )
    rows.append({
        "date": datetime.now().date(),
        "time": datetime.now().time().strftime("%H:%M:%S"),
        "test_case": test_case_name,
        "total_profit": total_profit,
        "job_details": " | ".join(details),
    })
    df = pd.DataFrame(rows)
    if os.path.exists(csv_file):
        df.to_csv(csv_file, mode="a", index=False, header=False)
    else:
        df.to_csv(csv_file, index=False, header=True)

def save_results_txt(test_case_name, output_folder="results"):
    global scheduled_jobs, total_profit
    os.makedirs(output_folder, exist_ok=True)
    path = os.path.join(output_folder, f"{test_case_name}.txt")
    with open(path, "w") as f:
        for job in sorted(scheduled_jobs, key=lambda x: x["id"]):
            if job["assigned_slots"]:
                f.write(",".join(map(str, job["assigned_slots"])) + "\n")
            else:
                f.write("null\n")
        f.write(str(total_profit) + "\n")
    print(f"\nResults saved to {path}")

# ---------------------------
# Main online algorithm (dynamic-score policy)
# ---------------------------
def run_online_algorithm_from_file(input_file):
    """
    Online preemptive scheduling with dynamic score:
      score_t = (w + l)^A / (p^B * frac_time_left(t)^C * frac_work_left(t)^D).
    At each integer time t, among jobs with r <= t <= d and remaining > 0,
    pick the job with maximum current score; break ties by higher w, earlier d, smaller id.
    """
    global calendar, scheduled_jobs, total_profit
    calendar = {}
    scheduled_jobs = []
    total_profit = 0

    jobs = read_jobs(input_file)

    # Preprocess
    for job in jobs:
        mark_infeasible(job)
        annotate_job(job)

    # Immediate penalties for infeasible on arrival
    for job in jobs:
        if not job["feasible"]:
            job["rejected"] = True
            total_profit -= job["l"]
            print(f"Job {job['id']} infeasible → -{job['l']}, slots = null")

    scheduled_jobs = jobs[:]  # keep for final writeout

    feasible = [j for j in jobs if j["feasible"] and not j["rejected"]]
    if not feasible:
        finalize_and_save(input_file)
        return

    T_min = min(j["r"] for j in feasible)
    T_max = max(j["d"] for j in feasible)

    # Group by release for O(1) arrivals
    releases = {}
    for j in feasible:
        releases.setdefault(j["r"], []).append(j)

    active = []  # simple list; we recompute scores each step for clarity

    for t in range(T_min, T_max + 1):
        # Add newly released jobs
        for j in releases.get(t, []):
            if j["remaining"] > 0:
                active.append(j)

        # Remove expired / finished from active
        active = [j for j in active if j["remaining"] > 0 and t <= j["d"]]

        # Pick job with max dynamic score (tie-breakers: w desc, d asc, id asc)
        if active:
            # Compute scores
            best = None
            best_key = None
            for j in active:
                s = dynamic_score(j, t)
                # We maximize (s, w, -d, -id) effectively
                key = (s, j["w"], -j["d"], -j["id"])
                if (best is None) or (key > best_key):
                    best = j
                    best_key = key
            # Execute one unit on the chosen job
            best["assigned_slots"].append(t)
            best["remaining"] -= 1
            calendar[t] = best["id"]
        else:
            calendar[t] = 0  # idle

    # Settle rewards/penalties
    for job in jobs:
        if job["rejected"]:
            continue
        if not job["feasible"]:
            continue
        if job["remaining"] == 0:
            total_profit += job["w"]
            print(f"Job {job['id']} DONE → +{job['w']}, slots = {job['assigned_slots']}")
        else:
            total_profit -= job["l"]
            print(f"Job {job['id']} NOT done → -{job['l']}, slots = {job['assigned_slots'] if job['assigned_slots'] else 'null'}")

    finalize_and_save(input_file)

def finalize_and_save(input_file):
    base = os.path.splitext(os.path.basename(input_file))[0]
    test_case_name = f"{base}_online_dynscore"
    base_for_opt = base.replace('_online','').replace('_offline','')
    optimal = optimal_profits.get(base_for_opt, 'N/A')
    print(f"\nFinal total profit: {total_profit} | Optimal: {optimal}")
    onlineprofit_abbas2.append(total_profit)

    log_results_csv(test_case_name)
    save_results_txt(test_case_name)

# ---------------------------
# Example usage
# ---------------------------
if __name__ == "__main__":
    tests = os.listdir("C:/Users/Collin/Documents/Universiteit Utrecht/Periode 1/Algorithms for Decision Support/Project/job_scheduling_instances")

    for input_file in tests:
        run_online_algorithm_from_file(f'job_scheduling_instances/{input_file}')
        print("\n" + "-"*50 + "\n")


Job 1 DONE → +64, slots = [12, 13, 14, 15, 16]
Job 2 DONE → +39, slots = [25, 26, 34, 35, 36, 37, 38, 39]
Job 3 DONE → +21, slots = [27, 28]
Job 4 DONE → +98, slots = [29, 30, 31, 32, 33]
Job 5 DONE → +32, slots = [62, 63, 64, 65]
Job 6 DONE → +97, slots = [4, 5, 6, 7, 8, 9, 10]
Job 7 DONE → +62, slots = [17, 18, 19, 20, 21]

Final total profit: 413 | Optimal: N/A

Results saved to results\instance_0001_online_dynscore.txt

--------------------------------------------------

Job 1 DONE → +72, slots = [30, 31, 32, 33]
Job 2 NOT done → -35, slots = [63, 64, 65, 69, 70, 71, 72, 73, 74]
Job 3 DONE → +32, slots = [91, 92, 93, 94]
Job 4 DONE → +78, slots = [82, 83, 84]
Job 5 DONE → +44, slots = [6, 7, 8, 9, 10]
Job 6 NOT done → -29, slots = [66, 67, 68]
Job 7 NOT done → -19, slots = [85, 86]
Job 8 DONE → +72, slots = [3, 4, 5]
Job 9 DONE → +65, slots = [50, 51, 52, 53, 54, 55, 56, 57]
Job 10 DONE → +94, slots = [34, 35, 36, 37, 38, 39]
Job 11 DONE → +74, slots = [60, 61, 62]
Job 12 DONE → +1

In [12]:
import os
from datetime import datetime
import pandas as pd
import heapq
from read_file import read_jobs

# ---------------------------
# Test instance optimal profits (optional, for printout)
# ---------------------------
optimal_profits = {
    "test1": 133,
    "test2": 44,
    "test3": 30,
    "test4": 10,
    "test5": 0,
    "test6": 130,
    "test7": 70
}

# ---------------------------
# Online scheduler state
# ---------------------------
calendar = {}           # t -> job_id (or 0 if idle)
scheduled_jobs = []     # list of job dicts with annotations
total_profit = 0

# ---------------------------
# Helpers
# ---------------------------
def mark_infeasible(job):
    job["feasible"] = (job["d"] - job["r"] + 1) >= job["p"]
    return job

def compute_score(job):
    # density-like priority; used for selection
    job["score"] = (job["w"] + job["l"]) / job["p"] if job["feasible"] else -1
    return job

def annotate_job(job):
    # fields used by the simulator
    job["assigned_slots"] = []
    job["remaining"] = job["p"] if job["feasible"] else 0
    job["rejected"] = False          # infeasible-at-arrival rejection
    job["penalized_now"] = False     # to avoid double-penalizing
    return job

def log_results_csv(test_case_name, csv_file="results_log.csv"):
    global scheduled_jobs, total_profit
    rows = []
    job_details = []
    for job in scheduled_jobs:
        slots = ",".join(map(str, job["assigned_slots"])) if job["assigned_slots"] else "null"
        job_details.append(f"id:{job['id']} r:{job['r']} d:{job['d']} p:{job['p']} w:{job['w']} l:{job['l']} slots:{slots}")
    rows.append({
        "date": datetime.now().date(),
        "time": datetime.now().time().strftime("%H:%M:%S"),
        "test_case": test_case_name,
        "total_profit": total_profit,
        "job_details": " | ".join(job_details)
    })
    df = pd.DataFrame(rows)
    if os.path.exists(csv_file):
        df.to_csv(csv_file, mode="a", index=False, header=False)
    else:
        df.to_csv(csv_file, index=False, header=True)

def save_results_txt(test_case_name, output_folder="results"):
    global scheduled_jobs, total_profit
    os.makedirs(output_folder, exist_ok=True)
    path = os.path.join(output_folder, f"{test_case_name}.txt")
    with open(path, "w") as f:
        for job in sorted(scheduled_jobs, key=lambda x: x["id"]):
            if job["assigned_slots"]:
                f.write(",".join(map(str, job["assigned_slots"])) + "\n")
            else:
                f.write("null\n")
        f.write(str(total_profit) + "\n")
    print(f"\nResults saved to {path}")

# ---------------------------
# Main online algorithm
# ---------------------------
def run_online_algorithm_from_file(input_file):
    """
    Online preemptive policy:
      - Time ticks t from min(r) to max(d).
      - At each t, consider jobs with r <= t <= d and remaining > 0.
      - Pick job with highest score = (w + l) / p. (Break ties by: higher w, earlier d, smaller id.)
      - Assign 1 unit at time t to that job (preemption allowed).
      - At the end: +w if remaining == 0; otherwise -l. Infeasible at arrival => immediate -l.
    """
    global calendar, scheduled_jobs, total_profit
    calendar = {}
    scheduled_jobs = []
    total_profit = 0

    # Load jobs
    jobs = read_jobs(input_file)

    # Preprocess jobs
    for job in jobs:
        mark_infeasible(job)
        compute_score(job)
        annotate_job(job)

    # Immediately reject infeasible jobs with penalty
    for job in jobs:
        if not job["feasible"]:
            job["rejected"] = True
            job["penalized_now"] = True
            total_profit -= job["l"]
            print(f"Job {job['id']} infeasible → -{job['l']}, slots = null")

    # If all feasible jobs are rejected, we still want to output their null lines later
    scheduled_jobs = jobs[:]  # keep original order for final output

    # Determine simulation horizon
    feasible_jobs = [j for j in jobs if j["feasible"] and not j["rejected"]]
    if feasible_jobs:
        T_min = min(j["r"] for j in feasible_jobs)
        T_max = max(j["d"] for j in feasible_jobs)
    else:
        # nothing schedulable; just finalize and save
        finalize_and_save(input_file)
        return

    # Active heap: max-heap by score (use negatives for heapq). Tie-breakers: -w, d, id
    # Entry = (-score, -w, d, id, job_ref)
    active = []
    # Jobs keyed by release times for efficient insertion
    releases = {}
    for j in feasible_jobs:
        releases.setdefault(j["r"], []).append(j)

    # Simulation over time
    for t in range(T_min, T_max + 1):
        # Add newly released jobs
        for j in releases.get(t, []):
            if j["remaining"] > 0:
                heapq.heappush(active, (-j["score"], -j["w"], j["d"], j["id"], j))

        # Drop expired or finished jobs from the top as needed
        while active and (active[0][4]["remaining"] == 0 or t > active[0][4]["d"]):
            heapq.heappop(active)

        # Also lazily skip expired/finished entries when popped later

        # Pick best available job (if any)
        chosen_job = None
        while active:
            _, _, _, _, cand = heapq.heappop(active)
            if cand["remaining"] > 0 and t <= cand["d"]:
                chosen_job = cand
                break
            # else skip finished/expired stales and keep popping

        if chosen_job is not None:
            # Assign one unit at time t
            chosen_job["assigned_slots"].append(t)
            chosen_job["remaining"] -= 1
            calendar[t] = chosen_job["id"]
            # If still has remaining and deadline not yet passed, push back for future consideration
            if chosen_job["remaining"] > 0 and t < chosen_job["d"]:
                heapq.heappush(active, (-chosen_job["score"], -chosen_job["w"], chosen_job["d"], chosen_job["id"], chosen_job))
        else:
            # Idle
            calendar[t] = 0

    # Settle rewards/penalties
    for job in jobs:
        if job["rejected"]:
            # already penalized
            continue
        if not job["feasible"]:
            # already handled as rejected
            continue
        if job["remaining"] == 0:
            total_profit += job["w"]
            print(f"Job {job['id']} DONE → +{job['w']}, slots = {job['assigned_slots']}")
        else:
            total_profit -= job["l"]
            print(f"Job {job['id']} NOT done → -{job['l']}, slots = {job['assigned_slots'] if job['assigned_slots'] else 'null'}")

    finalize_and_save(input_file)

def finalize_and_save(input_file):
    # Pretty print + persist
    base = os.path.splitext(os.path.basename(input_file))[0]
    test_case_name = f"{base}_online_highscore"
    base_for_opt = base.replace('_online','').replace('_offline','')
    optimal = optimal_profits.get(base_for_opt, 'N/A')
    print(f"\nFinal total profit: {total_profit} | Optimal: {optimal}")
    onlineprofit_abbas.append(total_profit)

    log_results_csv(test_case_name)
    save_results_txt(test_case_name)

# ---------------------------
# Example usage
# ---------------------------
if __name__ == "__main__":
    tests = os.listdir("C:/Users/Collin/Documents/Universiteit Utrecht/Periode 1/Algorithms for Decision Support/Project/job_scheduling_instances")

    for input_file in tests:
        run_online_algorithm_from_file(f'job_scheduling_instances/{input_file}')
        print("\n" + "-"*50 + "\n")


Job 1 DONE → +64, slots = [12, 13, 14, 15, 16]
Job 2 DONE → +39, slots = [25, 26, 34, 35, 36, 37, 38, 39]
Job 3 DONE → +21, slots = [27, 28]
Job 4 DONE → +98, slots = [29, 30, 31, 32, 33]
Job 5 DONE → +32, slots = [62, 63, 64, 65]
Job 6 DONE → +97, slots = [4, 5, 6, 7, 8, 9, 10]
Job 7 DONE → +62, slots = [17, 18, 19, 20, 21]

Final total profit: 413 | Optimal: N/A

Results saved to results\instance_0001_online_highscore.txt

--------------------------------------------------

Job 1 DONE → +72, slots = [30, 31, 32, 33]
Job 2 NOT done → -35, slots = [53, 54, 69, 70, 71]
Job 3 DONE → +32, slots = [90, 91, 92, 93]
Job 4 DONE → +78, slots = [77, 78, 79]
Job 5 DONE → +44, slots = [6, 7, 8, 9, 10]
Job 6 NOT done → -29, slots = [57, 64, 65, 66, 67, 68]
Job 7 DONE → +13, slots = [81, 82, 83, 84, 85]
Job 8 DONE → +72, slots = [3, 4, 5]
Job 9 DONE → +65, slots = [44, 46, 47, 48, 49, 50, 51, 52]
Job 10 DONE → +94, slots = [34, 35, 36, 37, 38, 39]
Job 11 DONE → +74, slots = [58, 59, 60]
Job 12 DONE

In [ ]:
#print(offlineprofit)
#print(onlineprofit)
#print(onlineprofit_abbas)
#print(onlineprofit_abbas2)

difflist_Teymur = list()
difflist_Abbas = list()
difflist_Abbas2 = list()


for x, y in zip(offlineprofit, onlineprofit):
    difflist_Teymur.append(x-y)

#print(difflist_Teymur)

for i in difflist_Teymur:
    if i < 0:
        print("PROBLEM!")
print("Teymur: ", sum(difflist_Teymur))        
###############################################        
difflist_Abbas = list()
for x, y in zip(offlineprofit, onlineprofit_abbas):
    difflist_Abbas.append(x-y)

#print(difflist_Abbas)

for i in difflist_Abbas:
    if i < 0:
        print("PROBLEM!")
print("Abbas: ", sum(difflist_Abbas))        
###############################################        
difflist_Abbas2 = list()
for x, y in zip(offlineprofit, onlineprofit_abbas2):
    difflist_Abbas2.append(x-y)

#print(difflist_Abbas2)

for i in difflist_Abbas2:
    if i < 0:
        print("PROBLEM!")
print("Abbas2: ", sum(difflist_Abbas2))    
###############################################
print("total reward max = ", sum(offlineprofit))


Teymur:  109030
Abbas:  70045
Abbas2:  59623
total reward max =  535020
